In [1]:
import requests
import pandas as pd
import os
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

import seaborn as sns
sns.set_context('poster')
sns.set(rc={'figure.figsize': (16., 9.)})
sns.set_style('whitegrid')

import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

In [2]:
API = "https://agridata.ec.europa.eu/extensions/DataPortal/API_Documentation.html"

In [3]:
foods = ["beef","pigmeat","sheepAndGoat","rawMilk","dairy","fruitAndVegetable","cereal","rice","oilseeds","sugar","oliveOil","wine"]

In [4]:
data={}
date = []
price = []
prod = []
var = []
veg = []
shandgoat=[]
for food in foods:
    url = f"https://www.ec.europa.eu/agrifood/api/{food}/prices?memberStateCodes=ES&beginDate=01/01/2001"
    resp = requests.get(url).json()

    for i in resp:
        try:
            date.append(i['beginDate'])
        except: #Rice doesn't have beginDate but has ym which means year month
            date.append(i['ym'])
        price.append(float(i['price'][1:].replace(",",".")))
        try:
            var.append(i['productName'])
        except:
            var.append("unknown")
        try:
            veg.append(i['product'])
        except:
            veg.append('unknown')
        try:
            shandgoat.append(i['category'])
        except:
            shandgoat.append('unknown')
        prod.append(food)

    data['product'] = prod
    data['date'] = date
    data['price'] = price
    data['variety'] = var
    data['veg'] = veg
    data['shandgoat'] = shandgoat

In [5]:
resp = pd.DataFrame(resp)
if not os.path.exists(f"../data/"):
    os.makedirs(f"../data/")
now = str(datetime.now())[:19].replace(":","_")
resp.to_csv(f"../data/api_raw.csv")

In [6]:
data = pd.DataFrame(data)
data.date = pd.to_datetime(data.date)
data.index = data.date
data = data.sort_index()

In [7]:
o = []
for i, row in data.iterrows():
    if row['variety'] != 'unknown':
        o.append(row['variety'].replace(" ","_").lower())
        
    elif row['veg'] != 'unknown':
        o.append(row['veg'].replace(" ","_").lower())
    elif row['shandgoat']!='unknown':
        o.append(row['shandgoat'].replace(" ","_").lower())

    else:
        o.append(row['product'].replace(" ","_").lower())
data ['product'] = o

In [8]:
price = []
for i, row in data.iterrows():
    if row['price'] != 0.00 or row['price'] != 0.0:
        price.append(row['price'])
    else:
        price.append(np.nan)
data['price'] = price
data.dropna(how='any',inplace=True)

In [9]:
# Translating product names to simplified Spanish to use them as keywords to search later
names = {'abricots':'albaricoque',
 'apples':'manzana',
 'asparagus':'espárrago',
 'beans':'judías',
 'beef':'ternera',
 'butter':'mantequilla',
 'cabbages':'col',
 'carrots':'zanahoria',
 'cauliflowers':'coliflor',
 'cherries':'cereza',
 'clementines':'clementina',
 'courgettes':'calabacín',
 'crude_olive-pomace_oil_(from_5_to_10%)':'aceite de orujo de oliva crudo de 5 a 10%',
 'crude_soya_bean_oil':'salsa de soja',
 'crude_sunflower_oil':'aceite de girasol',
 'cucumbers': 'pepino',
 'durum_wheat':'harina de trigo candeal',
 'edam':'queso edam',
 'egg_plants,_aubergines':'berenjena',
 'emmental':'queso emmental',
 'extra_virgin_olive_oil_(up_to_0.8%)':'aceite de oliva',
 'feed_barley':'cebada',
 'garlic':'ajo',
 'heavy_lamb':'cordero pascual (más de 13kg)',
 'lampante_olive_oil_(2%)':'aceite de oliva',
 'leeks':'puerro',
 'lemons':'limón',
 'lettuces':'lechuga',
 'light_lamb':'cordero lechal y recental (menos de 13kg)',
 'maize':'maíz',
 'malting_barley':'malta',
 'mandarins':'mandarina común',
 'melons':'melón',
 'milling_wheat':'harina de trigo',
 'mushrooms,_cultivated':'champiñón',
 'nectarines':'nectarina',
 'olive-pomace_oil_(up_to_1%)':'aceite de orujo de oliva (hasta 1%)',
 'onions':'cebolla',
 'oranges':'naranja',
 'organic_raw_milk':'leche entera orgánica',
 'peaches':'melocotón',
 'pears':'pera',
 'peppers':'pimiento',
 'pigmeat':'cerdo',
 'plums':'ciruela',
 'rapeseed':'colza',
 'raw_milk':'leche entera',
 'refined_olive-pomace_oil_(up_to_0.3%)':'aceite de orujo de oliva refinado (hasta 3%)',
 'refined_olive_oil_(up_to_0.3%)':'aceite de oliva',
 'rice':'arroz',
 'satsumas':'mandarina satsuma',
 'sheepandgoat':'cabra y oveja',
 'smp':'leche semidesnatada en polvo',
 'soya_meal':'harina de soja',
 'strawberries':'fresa',
 'sugar':'azúcar',
 'sunflower_seed':'pipas de girasol',
 'sunflower_seed_meal':'harina de pipas de girasol',
 'table_grapes':'uva',
 'tomatoes':'tomate',
 'virgin_olive_oil_(up_to_2%)':'aceite de oliva',
 'water_melons':'sandía',
 'wheypowder':'suero de leche',
 'wine':'vino',
 'wmp':'leche entera en polvo'}

In [10]:
data['product'] = data['product'].map(names)

In [11]:
data.drop(['date','variety','veg','shandgoat'], axis=1, inplace=True)

In [12]:
for i in sorted(list(set(data['product']))):
    x = len(data[data['product']==i])
    print(f"{i} has {x} rows")

aceite de girasol has 95 rows
aceite de oliva has 14106 rows
aceite de orujo de oliva (hasta 1%) has 6 rows
aceite de orujo de oliva crudo de 5 a 10% has 1966 rows
aceite de orujo de oliva refinado (hasta 3%) has 1958 rows
ajo has 1652 rows
albaricoque has 202 rows
arroz has 3269 rows
azúcar has 180 rows
berenjena has 765 rows
calabacín has 769 rows
cebada has 436 rows
cebolla has 545 rows
cerdo has 4047 rows
cereza has 176 rows
champiñón has 841 rows
ciruela has 222 rows
clementina has 376 rows
col has 41 rows
coliflor has 742 rows
colza has 48 rows
cordero lechal y recental (menos de 13kg) has 362 rows
cordero pascual (más de 13kg) has 362 rows
espárrago has 111 rows
fresa has 294 rows
harina de pipas de girasol has 47 rows
harina de soja has 44 rows
harina de trigo has 441 rows
harina de trigo candeal has 356 rows
judías has 720 rows
leche entera has 203 rows
leche entera en polvo has 1 rows
leche entera orgánica has 10 rows
leche semidesnatada en polvo has 264 rows
lechuga has 788 

In [13]:
#Dropping rare products and data with less than 50 records 
gooddata = []
for i, row in data.iterrows():
    if row['product']=='leche entera en polvo':
        gooddata.append(np.nan)
    elif row['product']=='leche entera orgánica':
        gooddata.append(np.nan)
    elif row['product']=='aceite de orujo de oliva (hasta 1%)':
        gooddata.append(np.nan)
    elif row['product']=='aceite de orujo de oliva crudo de 5 a 10%':
        gooddata.append(np.nan)
    elif row['product']=='aceite de orujo de oliva refinado (hasta 3%)':
        gooddata.append(np.nan)    
    elif row['product']=='col':
        gooddata.append(np.nan)
    elif row['product']=='colza':
        gooddata.append(np.nan)
    elif row['product']=='harina de soja':
        gooddata.append(np.nan)
    elif row['product']=='salsa de soja':
        gooddata.append(np.nan)
    else:
        gooddata.append(row['product'])
data['product'] = gooddata
data.dropna(how='any',inplace=True)

In [16]:
data.sample(50)

,product,price
date,,
2013-03-25,mandarina común,71.50
2008-08-25,ciruela,101.54
2008-10-20,tomate,207.31
2020-10-08,pera,99.00
2014-03-31,naranja,44.50
2015-09-28,aceite de oliva,380.50
2020-05-18,ternera,355.51
2013-06-17,manzana,92.02
2003-01-20,queso edam,568.97


In [15]:
now = str(datetime.now())[:19].replace(":","_")
if not os.path.exists(f"../mydata/cleandata/{now}"):
    os.makedirs(f"../mydata/cleandata/{now}")
data.to_csv(f"../mydata/cleandata/{now}/data_{now}.csv")
data.to_csv(f"../mydata/cleandata/data.csv")